In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
# Setting up the URLs to scrape from.
urls = ['https://www.basketball-reference.com/play-index/tgl_finder.cgi?request=1&match=game&lg_id=NBA&is_playoffs=N&team_seed_cmp=eq&opp_seed_cmp=eq&year_min=2011&year_max=2020&is_range=N&game_num_type=team&order_by=date_game']

# Running a loop to go over all pages of the table we scrape the data from.
qq = 'https://www.basketball-reference.com/play-index/tgl_finder.cgi?request=1&player=&match=game&lg_id=NBA&year_min=2011&year_max=2020&team_id=&opp_id=&is_range=N&is_playoffs=N&round_id=&best_of=&team_seed=&opp_seed=&team_seed_cmp=eq&opp_seed_cmp=eq&game_num_type=team&game_num_min=&game_num_max=&game_month=&game_location=H&game_result=&is_overtime=&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=date_game&order_by_asc=&offset='
for i in range (100,11701,100):
    y=str(i)
    urls.append(qq+y)

# Scraping table headers (aka labels) from the table
r_bbref_3d = rq.get(urls[0])
stats_bbref_soup = BeautifulSoup(r_bbref_3d.text, 'lxml')

look_for_headers = stats_bbref_soup.table.thead.find_all('tr')
look_for_headers[:1]
headers_list = look_for_headers[1].text
headers_list = headers_list.split('\n')
headers_list = headers_list[2:-1]

# Adding 'OPP' to opponent statistics labels
for i in range(19,32):
    headers_list[i] = 'OPP '+headers_list[i]

# Scraping all the data from the table, row by row
team_stats_byrow = []
temp = []

for url in urls:
    r_bbref_3d = rq.get(url)
    stats_bbref_soup = BeautifulSoup(r_bbref_3d.text, 'lxml')
            
    every_row = stats_bbref_soup.table.tbody.find_all('tr')  
    for row in every_row:
        list_of_stats = row.find_all('td')
        for td in list_of_stats:
            stat = td.text
            temp.append(stat)
        team_stats_byrow.append(temp)
        temp = []

# Creating a data frame for all scraped data
teamStats_df = pd.DataFrame(data=team_stats_byrow, columns=headers_list)
teamStats_df = teamStats_df.drop(teamStats_df.columns[2], axis=1)
teamStats_df = teamStats_df.dropna()

teamStats_df[['MP', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 'FT', 'FTA', 'PTS', 'OPP FG', 'OPP FGA', 'OPP 2P', 'OPP 2PA', 'OPP 3P', 'OPP 3PA', 'OPP FT', 'OPP FTA', 'OPP PTS']] = teamStats_df[['MP', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 'FT', 'FTA', 'PTS', 'OPP FG', 'OPP FGA', 'OPP 2P', 'OPP 2PA', 'OPP 3P', 'OPP 3PA', 'OPP FT', 'OPP FTA', 'OPP PTS']].astype(int)
teamStats_df[['FG%', '2P%', '3P%', 'FT%', 'OPP FG%', 'OPP 2P%', 'OPP 3P%', 'OPP FT%']] = teamStats_df[['FG%', '2P%', '3P%', 'FT%', 'OPP FG%', 'OPP 2P%', 'OPP 3P%', 'OPP FT%']].astype(float)
teamStats_df['Date'] = pd.to_datetime(teamStats_df['Date'])

teamStats_df

In [ ]:
# Exporting the dataframe to a csv file
teamStats_df.to_csv('NBA_Games_all_Decade.csv')